PDF 파일기반 질의응답 챗봇 (랭체인, 그라디오, Upstage)

In [1]:
# %pip install -q openai
# %pip install -q langchain
# %pip install -q -U langchain-openai

In [2]:
from dotenv import load_dotenv
import os

# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:4])

UPSTAGE_API_KEY = os.getenv("UPSTAGE_API_KEY")
print(UPSTAGE_API_KEY[30:])

sk-p
24


In [3]:
# %pip install -q pypdf
# %pip install -q faiss-cpu
# %pip install -q tiktoken

In [4]:
from langchain.document_loaders import PyPDFLoader
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

In [5]:
# PDF 로드
loader = PyPDFLoader("../data/tutorial-korean.pdf")
documents = loader.load()
print(len(documents))

39


In [6]:
# 텍스트 분할
#text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200, 
    chunk_overlap=0,
    separators=["\n\n", "\n", ".", "!", "?", ",", " ", ""]
)
#texts = text_splitter.split_documents(documents)
texts = loader.load_and_split(text_splitter=text_splitter)
print(len(texts))

284


In [7]:
from langchain_upstage import UpstageEmbeddings

# OpenAI Embeddings 적용
#embeddings = OpenAIEmbeddings()
embeddings = UpstageEmbeddings(model="solar-embedding-1-large")

# FAISS 벡터 저장소 생성
vector_store = FAISS.from_documents(texts, embeddings)
vector_store.save_local("../db/faiss_db")
# 벡터 저장소에서 검색하기
# search_type="similarity" 옵션은 retrieval 객체에서 유사성 검색을 사용하여 질문 vector와 가장 유사한 문장 vector를 선택함
# search_kwargs 옵션은 vector 저장소에서 Prompt 2개의 텍스트 덩어리로 보내려는 것을 의미함
retriever = vector_store.as_retriever(
    search_type="similarity", 
    search_kwargs={"k": 6}
)

In [8]:
from langchain_openai import ChatOpenAI
from langchain_upstage import ChatUpstage
from langchain.chains import RetrievalQA

# ChatOpenAI는 기본모델인  gpt-3.5-turbo 사용하고
# temperature=0은 보수적인 지문에 대한 답변을 내고, temperature=1 다양한 답변을 낼 수 있음 
#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
llm = ChatUpstage(
        model="solar-pro",
        base_url="https://api.upstage.ai/v1",
        temperature=0.5
)

In [9]:
from langchain.prompts import PromptTemplate

# 한국어 최적화 프롬프트
prompt_template = """
당신은 BlueJ 프로그래밍 환경 전문가입니다. 
아래 문서 내용을 바탕으로 정확하고 친절한 답변을 제공해주세요.

문서 내용:
{context}

질문: {question}

답변 규칙:
1. 문서 내용만을 근거로 답변하세요
2. 단계별 설명이 필요하면 순서대로 작성하세요  
3. 구체적인 메뉴명, 버튼명을 포함하세요
4. 문서에 없는 정보는 "문서에서 찾을 수 없습니다"라고 하세요

답변:"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)
print(" 프롬프트 설정 완료")
print(prompt)

 프롬프트 설정 완료
input_variables=['context', 'question'] input_types={} partial_variables={} template='\n당신은 BlueJ 프로그래밍 환경 전문가입니다. \n아래 문서 내용을 바탕으로 정확하고 친절한 답변을 제공해주세요.\n\n문서 내용:\n{context}\n\n질문: {question}\n\n답변 규칙:\n1. 문서 내용만을 근거로 답변하세요\n2. 단계별 설명이 필요하면 순서대로 작성하세요  \n3. 구체적인 메뉴명, 버튼명을 포함하세요\n4. 문서에 없는 정보는 "문서에서 찾을 수 없습니다"라고 하세요\n\n답변:'


In [10]:
# RetrievalQA가 실질적인 RAG를 수행하는 객체
chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,  # 기존 retriever 유지
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True
)
chain

RetrievalQA(verbose=False, combine_documents_chain=StuffDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\n당신은 BlueJ 프로그래밍 환경 전문가입니다. \n아래 문서 내용을 바탕으로 정확하고 친절한 답변을 제공해주세요.\n\n문서 내용:\n{context}\n\n질문: {question}\n\n답변 규칙:\n1. 문서 내용만을 근거로 답변하세요\n2. 단계별 설명이 필요하면 순서대로 작성하세요  \n3. 구체적인 메뉴명, 버튼명을 포함하세요\n4. 문서에 없는 정보는 "문서에서 찾을 수 없습니다"라고 하세요\n\n답변:'), llm=ChatUpstage(client=<openai.resources.chat.completions.completions.Completions object at 0x11822d550>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x11822dbe0>, model_name='solar-pro', temperature=0.5, model_kwargs={}, upstage_api_key=SecretStr('**********'), upstage_api_base='https://api.upstage.ai/v1'), output_parser=StrOutputParser(), llm_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content

In [11]:
query = "코드패드는 무엇이고 어떻게 사용하나요?"
query_result = chain.invoke(query)
print(type(query_result))
print(query_result)

<class 'dict'>
{'query': '코드패드는 무엇이고 어떻게 사용하나요?', 'result': '**코드패드(Code Pad)란?**  \n코드패드는 BlueJ에서 자바 코드(표현식 또는 명령문)의 일부분을 빠르게 실행하고 결과를 확인할 수 있는 도구입니다. 프로그램의 동작을 조사하거나 코드 구문을 테스트할 때 유용합니다.  \n\n**코드패드 사용 방법** (단계별 설명):  \n\n1. **코드패드 열기**  \n   - BlueJ 메인 메뉴에서 `보기(Show)` 메뉴를 선택합니다.  \n   - 하위 메뉴에서 **코드패드 보기(Show Code Pad)**를 클릭합니다.  \n\n2. **코드 입력 및 실행**  \n   - 코드패드 영역에 실행하고자 하는 자바 표현식 또는 명령문을 입력합니다.  \n     *예시: `2 + 3`, `System.out.println("Hello")`*  \n   - 키보드 **Enter** 키를 눌러 코드를 실행합니다. 결과는 코드패드 하단에 표시됩니다.  \n\n3. **이전 입력 재사용**  \n   - 이전에 입력한 코드는 자동으로 저장됩니다.  \n   - **상화 화살표 키(↑/↓)**로 이전/다음 입력을 선택해 재실행하거나 수정할 수 있습니다.  \n\n4. **기능 요약**  \n   - **목적**: 코드 조각 테스트, 구문 검증, 프로그램 동작 분석.  \n   - **입력 방식**: 라인 단위로 Enter 키로 실행.  \n   - **역사 관리**: 화살표 키로 이전 입력 조회/수정 가능.  \n\n> 문서에서 언급되지 않은 기능(예: 단축키, 코드 저장 등)은 **"문서에서 찾을 수 없습니다"**로 답변드립니다.', 'source_documents': [Document(id='49e0b23b-1bf2-479e-85dd-9e93ff3e7b69', metadata={'producer': 'Acrobat Distiller with ezUniHFT', 'creator': 'PScript5.

In [12]:
query_result['result']

'**코드패드(Code Pad)란?**  \n코드패드는 BlueJ에서 자바 코드(표현식 또는 명령문)의 일부분을 빠르게 실행하고 결과를 확인할 수 있는 도구입니다. 프로그램의 동작을 조사하거나 코드 구문을 테스트할 때 유용합니다.  \n\n**코드패드 사용 방법** (단계별 설명):  \n\n1. **코드패드 열기**  \n   - BlueJ 메인 메뉴에서 `보기(Show)` 메뉴를 선택합니다.  \n   - 하위 메뉴에서 **코드패드 보기(Show Code Pad)**를 클릭합니다.  \n\n2. **코드 입력 및 실행**  \n   - 코드패드 영역에 실행하고자 하는 자바 표현식 또는 명령문을 입력합니다.  \n     *예시: `2 + 3`, `System.out.println("Hello")`*  \n   - 키보드 **Enter** 키를 눌러 코드를 실행합니다. 결과는 코드패드 하단에 표시됩니다.  \n\n3. **이전 입력 재사용**  \n   - 이전에 입력한 코드는 자동으로 저장됩니다.  \n   - **상화 화살표 키(↑/↓)**로 이전/다음 입력을 선택해 재실행하거나 수정할 수 있습니다.  \n\n4. **기능 요약**  \n   - **목적**: 코드 조각 테스트, 구문 검증, 프로그램 동작 분석.  \n   - **입력 방식**: 라인 단위로 Enter 키로 실행.  \n   - **역사 관리**: 화살표 키로 이전 입력 조회/수정 가능.  \n\n> 문서에서 언급되지 않은 기능(예: 단축키, 코드 저장 등)은 **"문서에서 찾을 수 없습니다"**로 답변드립니다.'

In [13]:
query_result['source_documents']

[Document(id='49e0b23b-1bf2-479e-85dd-9e93ff3e7b69', metadata={'producer': 'Acrobat Distiller with ezUniHFT', 'creator': 'PScript5.dll Version 5.2', 'creationdate': '2005-04-26T15:21:34+09:00', 'moddate': '2005-04-26T15:21:34+09:00', 'author': 'Owner', 'title': '<426C75654AC7D1B1DBC6A9C5E4B8AEBEF3B9AEBCAD283230292E687770>', 'source': '../data/tutorial-korean.pdf', 'total_pages': 39, 'page': 1, 'page_label': '2'}, page_content='6. 코드패드 사용하기 ··················································· ······························································· ········· 21'),
 Document(id='3827c8af-4fcd-4c50-a79c-b22db57b96d7', metadata={'producer': 'Acrobat Distiller with ezUniHFT', 'creator': 'PScript5.dll Version 5.2', 'creationdate': '2005-04-26T15:21:34+09:00', 'moddate': '2005-04-26T15:21:34+09:00', 'author': 'Owner', 'title': '<426C75654AC7D1B1DBC6A9C5E4B8AEBEF3B9AEBCAD283230292E687770>', 'source': '../data/tutorial-korean.pdf', 'total_pages': 39, 'page': 20, 'page_label': '21'}, page_

In [14]:
for key in query_result.keys():
    print(key)

query
result
source_documents


In [15]:
print(type(query_result['source_documents']))
type(query_result['source_documents'][0])

<class 'list'>


langchain_core.documents.base.Document

In [16]:
doc = query_result['source_documents'][0]
print(type(doc))
doc_dict = doc.model_dump()
print(type(doc_dict))
print(doc_dict)

<class 'langchain_core.documents.base.Document'>
<class 'dict'>
{'id': '49e0b23b-1bf2-479e-85dd-9e93ff3e7b69', 'metadata': {'producer': 'Acrobat Distiller with ezUniHFT', 'creator': 'PScript5.dll Version 5.2', 'creationdate': '2005-04-26T15:21:34+09:00', 'moddate': '2005-04-26T15:21:34+09:00', 'author': 'Owner', 'title': '<426C75654AC7D1B1DBC6A9C5E4B8AEBEF3B9AEBCAD283230292E687770>', 'source': '../data/tutorial-korean.pdf', 'total_pages': 39, 'page': 1, 'page_label': '2'}, 'page_content': '6. 코드패드 사용하기 ··················································· ······························································· ········· 21', 'type': 'Document'}


In [17]:
for k in doc_dict:
    print(k)

id
metadata
page_content
type


In [18]:
doc_dict['metadata']['title']

'<426C75654AC7D1B1DBC6A9C5E4B8AEBEF3B9AEBCAD283230292E687770>'

In [19]:
query = "애플릿을 만들고 실행하는 방법을 설명해주세요"
query_result = chain.invoke(query)
print(query_result)

{'query': '애플릿을 만들고 실행하는 방법을 설명해주세요', 'result': '문서 내용을 바탕으로 애플릿을 만들고 실행하는 방법을 단계별로 설명드리겠습니다.\n\n### 1. 애플릿 만들기\n1. BlueJ에서 **New Class** 대화상자를 엽니다.\n2. "타입을 선택할 수 있습니다"를 참고해 **애플릿(Applet)** 타입을 선택합니다.\n3. 클래스 이름과 패키지를 입력한 후 생성하여 기본 골격을 만듭니다.\n   - 생성된 클래스에는 애플릿 실행을 위한 기본 코드가 포함됩니다.\n\n### 2. 애플릿 실행하기\n1. **예제(examples) 디렉토리**에서 **appletdemo** 프로젝트를 엽니다.\n2. 프로젝트 내 애플릿 클래스를 컴파일합니다.\n3. 컴파일 후 애플릿을 실행합니다.\n   - 실행 방법은 문서에 구체적으로 명시되지 않았으나, 일반 클래스 실행과 유사한 방식으로 진행됩니다.\n\n### 3. 추가 참고 사항\n- 문서에는 애플릿 실행을 위한 **구체적인 메뉴명/버튼명**이나 **HTML 임베딩 방법**이 명시되어 있지 않습니다.\n- "애플릿 만들기"와 관련된 내용은 9.2절(33페이지) 및 11.6절(38페이지)에서 추가로 확인할 수 있으나, 현재 문서에는 해당 페이지 내용이 누락되어 있습니다.\n\n> 문서에 없는 정보(예: 애플릿 실행 버튼 위치, HTML 파일 생성 방법 등)는 "문서에서 찾을 수 없습니다"로 답변드립니다.', 'source_documents': [Document(id='30638e08-09bb-424a-891e-23caca699c40', metadata={'producer': 'Acrobat Distiller with ezUniHFT', 'creator': 'PScript5.dll Version 5.2', 'creationdate': '2005-04-26T15:21:34+09:00', 'moddate': '2005-04-26T15:21:34+09:00', 'author': 'Owner

In [20]:
print(query_result['result'])

문서 내용을 바탕으로 애플릿을 만들고 실행하는 방법을 단계별로 설명드리겠습니다.

### 1. 애플릿 만들기
1. BlueJ에서 **New Class** 대화상자를 엽니다.
2. "타입을 선택할 수 있습니다"를 참고해 **애플릿(Applet)** 타입을 선택합니다.
3. 클래스 이름과 패키지를 입력한 후 생성하여 기본 골격을 만듭니다.
   - 생성된 클래스에는 애플릿 실행을 위한 기본 코드가 포함됩니다.

### 2. 애플릿 실행하기
1. **예제(examples) 디렉토리**에서 **appletdemo** 프로젝트를 엽니다.
2. 프로젝트 내 애플릿 클래스를 컴파일합니다.
3. 컴파일 후 애플릿을 실행합니다.
   - 실행 방법은 문서에 구체적으로 명시되지 않았으나, 일반 클래스 실행과 유사한 방식으로 진행됩니다.

### 3. 추가 참고 사항
- 문서에는 애플릿 실행을 위한 **구체적인 메뉴명/버튼명**이나 **HTML 임베딩 방법**이 명시되어 있지 않습니다.
- "애플릿 만들기"와 관련된 내용은 9.2절(33페이지) 및 11.6절(38페이지)에서 추가로 확인할 수 있으나, 현재 문서에는 해당 페이지 내용이 누락되어 있습니다.

> 문서에 없는 정보(예: 애플릿 실행 버튼 위치, HTML 파일 생성 방법 등)는 "문서에서 찾을 수 없습니다"로 답변드립니다.


In [21]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA

system_template = """Use the following pieces of context to answer the user's question shortly.
Given the following summaries of a long document and a question, create a final answer with references ("SOURCES"), 
use "SOURCES" in capital letters regardless of the number of sources.
If you don't know the answer, just say that "I don't know", don't try to make up an answer.
----------------
{summaries}

You MUST answer in Korean and in Markdown format:
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("human", "{question}")
])

In [22]:
chain_type_kwargs = {
    "prompt": prompt,
    "document_variable_name": "summaries",
}

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)  # GPT-4를 사용하려면 model="gpt-4"

chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs,
    input_key="question"
)

In [23]:
query = "소비자 부문 AI 활용사례 무엇인가요?"
query_result = chain.invoke({"question": query})
print(type(query_result))
print(query_result)

<class 'dict'>
{'question': '소비자 부문 AI 활용사례 무엇인가요?', 'result': '**소비자 부문 AI 활용사례**에 대한 구체적인 내용은 제공되지 않았습니다. 더 자세한 정보를 원하신다면 문서의 해당 부분을 확인하시기 바랍니다.\n\n**SOURCES**\n- 11.5. 독립형 어플리케이션 생성\n- 8. 독립형 어플리케이션 생성\n- 3.4. 실행하기\n- 11. 요약 정리\n- 1.4. 피드백\n- 11.7. 기타 기능들', 'source_documents': [Document(id='ba02069b-6e40-4c9d-82f5-0c4a9a6037c4', metadata={'producer': 'Acrobat Distiller with ezUniHFT', 'creator': 'PScript5.dll Version 5.2', 'creationdate': '2005-04-26T15:21:34+09:00', 'moddate': '2005-04-26T15:21:34+09:00', 'author': 'Owner', 'title': '<426C75654AC7D1B1DBC6A9C5E4B8AEBEF3B9AEBCAD283230292E687770>', 'source': '../data/tutorial-korean.pdf', 'total_pages': 39, 'page': 2, 'page_label': '3'}, page_content='11.5.  독립형 어플리케이션 생성 ······································· ······················································ 38'), Document(id='f10e98a8-6d00-4179-a8b1-85dbbf1b19f6', metadata={'producer': 'Acrobat Distiller with ezUniHFT', 'creator': 'PScript5.dll Version 5.2', 'creationdate': '2

In [24]:
print(query_result['result'])

**소비자 부문 AI 활용사례**에 대한 구체적인 내용은 제공되지 않았습니다. 더 자세한 정보를 원하신다면 문서의 해당 부분을 확인하시기 바랍니다.

**SOURCES**
- 11.5. 독립형 어플리케이션 생성
- 8. 독립형 어플리케이션 생성
- 3.4. 실행하기
- 11. 요약 정리
- 1.4. 피드백
- 11.7. 기타 기능들


In [25]:
query_result['source_documents']

[Document(id='ba02069b-6e40-4c9d-82f5-0c4a9a6037c4', metadata={'producer': 'Acrobat Distiller with ezUniHFT', 'creator': 'PScript5.dll Version 5.2', 'creationdate': '2005-04-26T15:21:34+09:00', 'moddate': '2005-04-26T15:21:34+09:00', 'author': 'Owner', 'title': '<426C75654AC7D1B1DBC6A9C5E4B8AEBEF3B9AEBCAD283230292E687770>', 'source': '../data/tutorial-korean.pdf', 'total_pages': 39, 'page': 2, 'page_label': '3'}, page_content='11.5.  독립형 어플리케이션 생성 ······································· ······················································ 38'),
 Document(id='f10e98a8-6d00-4179-a8b1-85dbbf1b19f6', metadata={'producer': 'Acrobat Distiller with ezUniHFT', 'creator': 'PScript5.dll Version 5.2', 'creationdate': '2005-04-26T15:21:34+09:00', 'moddate': '2005-04-26T15:21:34+09:00', 'author': 'Owner', 'title': '<426C75654AC7D1B1DBC6A9C5E4B8AEBEF3B9AEBCAD283230292E687770>', 'source': '../data/tutorial-korean.pdf', 'total_pages': 39, 'page': 2, 'page_label': '3'}, page_content='8. 독립형 어플리케이션 생성

In [26]:
for doc in query_result['source_documents']:
    print('내용 : ' + doc.page_content[0:100].replace('\n', ' '))
    print('파일 : ' + doc.metadata['source'])
    print('페이지 : ' + str(doc.metadata['page']))

내용 : 11.5.  독립형 어플리케이션 생성 ······································· ·······································
파일 : ../data/tutorial-korean.pdf
페이지 : 2
내용 : 8. 독립형 어플리케이션 생성 ··············································· ···································
파일 : ../data/tutorial-korean.pdf
페이지 : 2
내용 : 3.4. 실행하기 ·················································· ·······································
파일 : ../data/tutorial-korean.pdf
페이지 : 1
내용 : 11.  요약 정리 ····················································· ···································
파일 : ../data/tutorial-korean.pdf
페이지 : 2
내용 : 1.4. 피드백 ···················································· ······································
파일 : ../data/tutorial-korean.pdf
페이지 : 1
내용 : 11.7.  기타 기능들 ··············································· ······································
파일 : ../data/tutorial-korean.pdf
페이지 : 2


In [27]:
bot_message = query_result['result']
for i, doc in enumerate(query_result['source_documents']):
    bot_message += '[' + str(i+1) + '] ' + doc.metadata['source'] + '(' + str(doc.metadata['page']) + ') '
    print(bot_message)

**소비자 부문 AI 활용사례**에 대한 구체적인 내용은 제공되지 않았습니다. 더 자세한 정보를 원하신다면 문서의 해당 부분을 확인하시기 바랍니다.

**SOURCES**
- 11.5. 독립형 어플리케이션 생성
- 8. 독립형 어플리케이션 생성
- 3.4. 실행하기
- 11. 요약 정리
- 1.4. 피드백
- 11.7. 기타 기능들[1] ../data/tutorial-korean.pdf(2) 
**소비자 부문 AI 활용사례**에 대한 구체적인 내용은 제공되지 않았습니다. 더 자세한 정보를 원하신다면 문서의 해당 부분을 확인하시기 바랍니다.

**SOURCES**
- 11.5. 독립형 어플리케이션 생성
- 8. 독립형 어플리케이션 생성
- 3.4. 실행하기
- 11. 요약 정리
- 1.4. 피드백
- 11.7. 기타 기능들[1] ../data/tutorial-korean.pdf(2) [2] ../data/tutorial-korean.pdf(2) 
**소비자 부문 AI 활용사례**에 대한 구체적인 내용은 제공되지 않았습니다. 더 자세한 정보를 원하신다면 문서의 해당 부분을 확인하시기 바랍니다.

**SOURCES**
- 11.5. 독립형 어플리케이션 생성
- 8. 독립형 어플리케이션 생성
- 3.4. 실행하기
- 11. 요약 정리
- 1.4. 피드백
- 11.7. 기타 기능들[1] ../data/tutorial-korean.pdf(2) [2] ../data/tutorial-korean.pdf(2) [3] ../data/tutorial-korean.pdf(1) 
**소비자 부문 AI 활용사례**에 대한 구체적인 내용은 제공되지 않았습니다. 더 자세한 정보를 원하신다면 문서의 해당 부분을 확인하시기 바랍니다.

**SOURCES**
- 11.5. 독립형 어플리케이션 생성
- 8. 독립형 어플리케이션 생성
- 3.4. 실행하기
- 11. 요약 정리
- 1.4. 피드백
- 11.7. 기타 기능들[1] ../data/tutorial-korean.pdf(2) [2] ../d

In [32]:
#!pip install gradio

In [33]:
from langchain.document_loaders import PyPDFLoader
from langchain_upstage import UpstageEmbeddings, ChatUpstage
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import ChatPromptTemplate
import gradio as gr

# 1. PDF를 한 번만 로드하여 벡터 저장소 생성
def initialize_retriever():
    pdf_path = "../data/tutorial-korean.pdf"
    # PDF 파일을 로드하여 문서 객체로 변환
    loader = PyPDFLoader(pdf_path)
    documents = loader.load()

    # 문서를 일정한 크기로 분할 (chunk_size=1000, 중첩 없음)
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=200, 
        chunk_overlap=0,
        separators=["\n\n", "\n", ".", "!", "?", ",", " ", ""]
    )
    texts = loader.load_and_split(text_splitter=text_splitter)

    # 문서의 텍스트를 벡터 임베딩으로 변환
    embeddings = UpstageEmbeddings(model="solar-embedding-1-large")

    # 변환된 벡터를 FAISS 벡터 저장소에 저장
    vector_store = FAISS.from_documents(texts, embeddings)
    vector_store.save_local("../db/faiss_db")
    # 저장된 벡터를 검색할 수 있는 retriever 생성 (유사한 문서 2개 검색)
    retriever = vector_store.as_retriever(
        search_type="similarity", 
        search_kwargs={"k": 6}
    )
    return retriever

# 2. 전역 retriever 생성 (앱 시작 시 한 번만 실행)
retriever = initialize_retriever()

retrieved_docs = retriever.invoke("코드패드는 무엇이고 어떻게 사용하나요?")
for doc in retrieved_docs:
    print(doc.page_content)

/Users/eon/skRookies/myLangChain/mylangchain-app/faiss-env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


6. 코드패드 사용하기 ··················································· ······························································· ········· 21
216. 코드패드의 사용
BlueJ 코드패드는 자바 코드(표현식과 명령문)의 일부분을 쉽고 빠르게 평가할 수 있는 기능을 
제공합니다. 따라서, 코드패드는 자바언어로 작성된 프로그램 코드의 의미를 상세히 조사하거나 구문을 예증하고 시험하는데 사용할 수 있 습니다.
6.1. 코드패드 나타내기
요 약 코드패드에 명령문들을 입력하여 실행시킬 수 있습니다.
코드패드에서는 사용자가 이전에 입력한 내용들에 대한 이력을 저장하고 있습니다. 상하 화살표를 
이용하여 이전에 입력한 내용을 손쉽게 다시 입력시킬 수 있을 뿐만 아니라, 이전에 입력했던 내용을 수정하여 재입력시킬 수도 있습니다.   요 약 입력이력을 사용하기 위해서는 상하 화살표를 사용하십시오.
6.1. 코드패드 보기 ················································ ······························································· ··········· 21
코드패드 영역에는 표현식 또는 문장들을 입력할 수 있습니다. 키보드의 Enter를 누르면 
입력된 표현식 또는 문장이 라인단위로 평가되어 그 결과가 화면에 나타납니다.
그림 13 : 코드패드를 나타낸 메인화면  요 약  코드패드를 사용하기 위해 보기메뉴에서 코드패드보기(Show Code Pad)를 
 선택하십시오


In [ ]:
# 3. 채팅 응답 함수 (retriever를 재사용)
def chat_respond(message, chat_history):
    #  시스템 메시지 템플릿: 문서 요약을 기반으로 질문에 답변하도록 설정
    system_template = """당신은 BlueJ 프로그래밍 환경 전문가입니다. 
        아래 문서 내용을 바탕으로 정확하고 친절한 답변을 제공해주세요.

        문서 내용:
        {context}

        질문: {question}

        답변 규칙:
        1. 문서 내용만을 근거로 답변하세요
        2. 단계별 설명이 필요하면 순서대로 작성하세요  
        3. 구체적인 메뉴명, 버튼명을 포함하세요
        4. 문서에 없는 정보는 "문서에서 찾을 수 없습니다"라고 하세요

        답변:
    """
    # 사용자 질문을 받아 최종 Prompt 구성
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_template),
        ("human", "{question}")
    ])

    chain_type_kwargs = {
        "prompt": prompt, #  LLM이 사용할 프롬프트 지정
        "document_variable_name": "context", #  문서 요약 데이터를 LLM에 전달할 변수 이름
    }
    
    llm = ChatUpstage(
        model="solar-pro",
        base_url="https://api.upstage.ai/v1",
        temperature=0.5
    )
    
    # RetrievalQA 체인 생성
    chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",  #  문서를 하나의 큰 텍스트로 처리하는 방식
        retriever=retriever, #  유사 문서를 검색하는 retriever 연결
        return_source_documents=True, #  답변에 참조한 문서 정보 포함
        chain_type_kwargs=chain_type_kwargs,
        input_key="question" #  사용자 질문을 "question" 키로 전달
    )
    
    # LLM을 호출하여 질문에 대한 응답 생성
    query_result = chain.invoke({"question": message})

    # 모델의 답변을 가져오기
    bot_message = query_result['result']

    # 참조한 문서 정보를 응답에 추가
    for i, doc in enumerate(query_result['source_documents']):
        bot_message += f' [{i+1}] {doc.metadata.get("source", "Unknown")} (Page {doc.metadata.get("page", "N/A")})'

    # Gradio 채팅 기록 형식에 맞춰 응답을 저장
    chat_history.append({"role": "user", "content": message})  # 사용자 메시지 추가
    chat_history.append({"role": "assistant", "content": bot_message})  # 봇 응답 추가
    
    # 입력 창 초기화 및 갱신된 채팅 기록 반환
    return "", chat_history

# 4. # Gradio UI 생성 및 실행
with gr.Blocks() as demo:  
    # 채팅 창 (채팅 메시지를 표시하는 Gradio 컴포넌트)
    chatbot = gr.Chatbot(label="채팅창", type="messages")
    # 사용자 입력 텍스트 박스 (메시지를 입력하는 필드)
    msg = gr.Textbox(label="입력")
    # 초기화 버튼 (채팅 기록을 초기화하는 버튼)
    clear = gr.Button("초기화")

    # 사용자가 입력 필드에 메시지를 입력하면 chat_respond()가 실행됨
    #  - 입력한 메시지는 msg에서 가져옴
    #  - 기존 채팅 기록(chatbot)과 함께 전달됨
    #  - 응답을 받은 후, msg를 초기화하고 chatbot에 대화 내용을 추가
    msg.submit(chat_respond, [msg, chatbot], [msg, chatbot])  
    # 초기화 버튼 클릭 시, 채팅 기록을 비우도록 설정
    clear.click(lambda: [], None, chatbot, queue=False)

# Gradio 앱 실행 (debug 모드 활성화)
demo.launch(debug=True)

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
